# A Bayesian approach to correlation for oceanographers

Correlations are a frequently used approach to understand the relationships between data series. Bayesian inference provides a framework for the estimation of correlation coefficients along with the uncertainty in this estimation. In this post I set out an example of how you can apply Bayesian methods to estimate correlation and uncertainty in your data.  I begin with an idealised example, before showing how it works for real ocean data.

First, we need to import some packages. These can be installed using pip or conda on your system.

In [1]:
import numpy as np
import holoviews as hv
import bokeh
from bokeh.plotting import figure, show

import nssh.nssh_mods.bayes_methods as bm

hv.notebook_extension('bokeh')
bokeh.io.output_notebook()

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Idealised example
We first look at an idealised example where we create a dataset that is generated using NumPy's built in random number generator.

In [3]:
N = 20 # Number of data points in our idealised dataset
means = np.array([ 0 , 0]) # Mean values for our bivariate normal distribution
variances = np.array([1, 1]) # Variances for our bivariate normal distribution
# We will generate datasets with correlation - which we call rho - that ranges from -1 to 1
rho = np.arange(-1,1,3e-2)
rho = rho[1:-1] # We exclude the end points of perfect correlation
data = np.empty( (N, 2, len( rho ) ) ) # Pre-allocate our idealised dataset
#We loop over the range of rhos to create an idealised dataset for each value of rho
for idx,r in enumerate( rho ):
    covs = np.array( [[variances[0]**2,r*variances[0]*variances[1]],[r*variances[0]*variances[1], variances[1]**2]])
    data[:,:,idx] = np.random.multivariate_normal( means, covs, N)
    

## Inspecting the data
As ever, the first thing to do is inspect our data. To do this, we use the holoviews package to scatterplot the data for each value of rho - you can use the slider to vary the values of rho.  

In [16]:
%%output size=130 # Set figure size
%%opts Scatter (size=10)
scatter_dict = {r:hv.Scatter(data[:,:,idx]) for idx,r in enumerate(rho)} #Create a key-value pair for each rho
scatter = hv.HoloMap(scatter_dict, kdims=['Rho']) #Create a HoloMap with this dictionary
scatter # Create the scatter plot

:HoloMap   [Rho]
   :Scatter   [x]   (y)

The scatter plot shows the variability that can occur, even when the data is drawn from a bivariate normal distribution!

## Infer the correlation using Bayesian methods
As we have the data, we can now proceed to develop a Bayesian model for the correlation.  [In a previous post](http://braaannigan.github.io/bayes_moc_gaussian.html) we looked at fitting a normal distribution to a dataset. The correlation inference proceeds on a similar basis, with the difference that we fit a bivariate normal distribution to a dataset:
$$ (x,y) \sim \textrm{MV}([\mu_{1},\mu_{2}],\Sigma)$$
where the $\mu_{i}$ values are the means and the covariance matrix
$$\Sigma = \begin{bmatrix}
    \sigma_{1}^{2} & \rho\sigma_{1}\sigma_{2} \\
     \rho\sigma_{1}\sigma_{2} & \sigma_{1}^{2} \\
\end{bmatrix}$$
Our job then is to infer the full set of parameters $\mu_{1},\mu_{2},\sigma_{1},\sigma_{2}$ and $\rho$, in order to get the latter correlation parameter that we are interested in.

We perform the inference separately for each dataset that corresponds to a different value of $\rho$. As well as inferring the parameters, we will also use the inference as a chance to generate some simulated data from our model.  We will use this simulated data to test our model below.

A word of warning: Bayesian inference is computationally expensive - performing the inference for 50-60 different values of $\rho$ can take a couple of hours.  You may want to carry out the inference for just a few value of $\rho$ to begin with.

rho_range = np.empty((len(rho),3)) # Pre-allocate an array to hold summary statstics for rho
trace_list = [] # A list to hold the full set of outputs from the inference
simulated_data = np.empty( (500,2,len(rho))) # An array to hold the data simulated under the model

for idx in np.arange(len(rho)): # Loop through each value of rho
    print(idx)
    trace,out_dict, ppc = bm.correlation(data[:,0,idx],data[:,1,idx]) # Carry out the inference
    trace_list.append(trace) # Add the trace to the trace_list
    del trace
    rho_range[idx,0] = out_dict['2.5'] # Calculate the summary statistics
    rho_range[idx,1] = out_dict['mean']
    rho_range[idx,2] = out_dict['97.5']
    simulated_data[:,:,idx] = ppc['mult_n'] # Save the simulated data for this value of rho

In [28]:
import pickle
#np.save('rho_range',rho_range)
#pickle.dump(trace_list,open("trace_list","wb"))
#np.save("simulated_data",simulated_data)
rho_range = np.load('rho_range.npy')
simulated_data = np.load('simulated_data.npy')
trace_list = pickle.load(open("trace_list","rb"));

We now define a function to create a plot using Bokeh showing the correlation and its uncertainty.

In [24]:
def fill_between(x, y, y_lower, y_upper,real_data, title = 'None', ylabel = None, xlabel = None):
    """Make a bokeh plot to show the an uncertainty range"""
    #Have to append a reversed series for the patch coordinates
    band_x = np.append(x, x[::-1])
    band_y = np.append(y_lower, y_upper[::-1])
    p = figure(title= title, height = 700, width = 700)
    p.line(x, y, line_width = 2)
    p.line(x, real_data, line_dash="4 4", line_width=1, color='gray')
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.5, line_color = "firebrick")
    p.xaxis.axis_label = xlabel
    p.yaxis.axis_label = ylabel
    p.xaxis.bounds = (-1,1)
    p.yaxis.bounds = (-1,1)
    show(p)

We call this plot using the $rho\_range$ array that contains the mean and uncertainty ranges.  We plot the true value of the correlation as a dashed line. As this is a Bokeh plot you can use the toolds on the right to zoom in.

In [29]:
fill_between(rho, rho_range[:,1], rho_range[:,0], rho_range[:,2], rho, title = 'Correlation and uncertainty', 
             xlabel = 'True Correlation', ylabel = 'Estimated Correlation')

The plot above shows the mean estimated value of the parameter in blue with the shaded area corresponding to the 95% credible range for the correlation. The dashed line shows the "true" parameter.

There is a clear bulge in the uncertainty for values of the true correlation close to 0. This model suggests that when true correlation is close to 0 you have a higher chance of inferring values of correlation that are a long way from their true value.

We can understand this in a probabilistic sense.  We can ask what the probability is of our estimate of correlation being out by more than 0.2. For example, this could mean that we estimate probability to be 0.3, when it is actually 0.5.  This is shown in the left plot below.  

We can also ask what the probability is of estimating the wrong sign of correlation due to variation in our data. For example, we might estimate the correlation to be -0.1 when the true value of 0.1. This is shown in the right panel below.

In [30]:
%%output size=150
bad_estimate =  np.empty(len(rho))
type_s =  np.empty(len(rho))

for idx in np.arange(len(rho)):
    bad_estimate[idx] = np.count_nonzero( np.abs(trace_list[idx]['r'] - rho[idx ])>0.2)/len(trace_list[idx]['r'])
    type_s[idx] = np.count_nonzero(np.sign(rho[idx]) != np.sign(trace_list[idx]['r']))/len(trace_list[idx]['r'])
magnitude = hv.Curve((rho, bad_estimate), kdims=['Rho'],vdims= ['Probability'], group='Probability of error greater than 0.2')
sign = hv.Curve((rho, type_s), kdims=['Rho'],vdims= ['Probability'], group='Probability of wrong sign')

magnitude + sign

:Layout
   .Probability_of_error_greater_than_0_full_stop_2.I :Curve   [Rho]   (Probability)
   .Probability_of_wrong_sign.I                       :Curve   [Rho]   (Probability)

The left hand plot confirms that there is a bulge around zero and the chances that our estimate of correlation might be off by a relevant amount is higher there.

The right hand plot shows that our chances of getting the wrong size of correlation are peaked around 0, as you would expect.  However, it also shows that the chances of getting the sign wrong are non-negligible even at values with magnitudes greater than 0.4!  This potential for large error is particularly important because publication bias means that outlier results are **more** likely to be published.

## Compare the "real" and simulated data
We can always fit a model to a dataset. The crucial step is to simulate data under the model to see if it is an adequate representation of the data and to understand how the model might be improved. The process of simulating data from Bayesian estimates of the parameters [was explained in a previous post](http://braaannigan.github.io/bayes_moc_gaussian.html). We generated this simulated data above when we did the inference and saved the 500 simulated data points for each value of $\rho$ in the array *simulated_data*. We plot this simulated data against the real data below.  You can use the slider to compare the results for different values of $\rho$.

In [33]:
%%output size=150
sim_dict = {r:hv.Scatter(simulated_data[:,:,idx]) for idx,r in enumerate(rho)} 
sim = hv.HoloMap(sim_dict, kdims=['Rho'])
sim*scatter


:HoloMap   [Rho]
   :Overlay
      .Scatter.I  :Scatter   [x]   (y)
      .Scatter.II :Scatter   [x]   (y)

The comparison of the real and simulated data confirms the impression we had from the range of uncertainty for $\rho$.  When the magnitude of the correlation is high - say $|\rho| > 0.5$, the simulated data and the real data all lie bunched tightly together.  However, when the correlation is lower we see that both the real data and the simulated data form more of a cloud. In this case the small sample size of the real dataset - just 20 independent points - means that randomness in sampling can lead to the magnitude of the correlation appearing much higher than it actually is.